In [30]:
import pandas as pd

In [31]:
# --- Load data ---
eurostat = pd.read_csv('/Users/weronikadorociak/Documents/LSE/MY498 Capstone Project/Coding/Capstone_Project/Data_cleaning/data/eurostat.csv')
nace = pd.read_csv('data/economic_activity_sector.csv')

In [33]:
# --- Prepare Eurostat aging data ---
selected_cols = ['age', 'sex', 'nace_r2', 'geo', 'TIME_PERIOD', 'OBS_VALUE']
eurostat = eurostat[selected_cols].copy()

In [34]:
eu_countries = [
    'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia',
    'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece',
    'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg',
    'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia',
    'Slovenia', 'Spain', 'Sweden'
]

In [35]:
# Filter relevant observations
eurostat = eurostat[
    eurostat["TIME_PERIOD"].isin([2014, 2024]) &
    (eurostat["sex"] == "Total") &
    (eurostat["age"].isin(["15 years or over", "50 years or over"])) &
    (eurostat["geo"].isin(eu_countries))
]

In [37]:
eurostat = eurostat.merge(nace, how='left', left_on='nace_r2', right_on = "Economic Activity")

,age,sex,nace_r2,geo,TIME_PERIOD,OBS_VALUE,Economic Activity,NACE Code,Sector (Section),Sector code,Sector
0,15 years or over,Total,"Crop and animal production, hunting and relate...",Austria,2014,186.5,"Crop and animal production, hunting and relate...",1.0,"A – Agriculture, forestry and fishing",A,"Agriculture, forestry and fishing"
1,15 years or over,Total,"Crop and animal production, hunting and relate...",Austria,2024,112.4,"Crop and animal production, hunting and relate...",1.0,"A – Agriculture, forestry and fishing",A,"Agriculture, forestry and fishing"
2,15 years or over,Total,"Crop and animal production, hunting and relate...",Belgium,2014,53.2,"Crop and animal production, hunting and relate...",1.0,"A – Agriculture, forestry and fishing",A,"Agriculture, forestry and fishing"
3,15 years or over,Total,"Crop and animal production, hunting and relate...",Belgium,2024,49.2,"Crop and animal production, hunting and relate...",1.0,"A – Agriculture, forestry and fishing",A,"Agriculture, forestry and fishing"
4,15 years or over,Total,"Crop and animal production, hunting and relate...",Bulgaria,2014,181.1,"Crop and animal production, hunting and relate...",1.0,"A – Agriculture, forestry and fishing",A,"Agriculture, forestry and fishing"
...,...,...,...,...,...,...,...,...,...,...,...
9823,50 years or over,Total,Unknown NACE activity,Sweden,2024,NaN,NaN,NaN,NaN,NaN,NaN
9824,50 years or over,Total,Unknown NACE activity,Slovenia,2014,NaN,NaN,NaN,NaN,NaN,NaN
9825,50 years or over,Total,Unknown NACE activity,Slovenia,2024,NaN,NaN,NaN,NaN,NaN,NaN
9826,50 years or over,Total,Unknown NACE activity,Slovakia,2014,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
eurostat = eurostat[['age', 'Sector', 'geo', 'TIME_PERIOD', 'OBS_VALUE']]
eurostat.columns = ['age', 'sector', 'country', 'year', 'num_of_workers']
eurostat["num_of_workers"] = eurostat["num_of_workers"] * 1000

excluded = ['No response', 'Total - all NACE activities', 'Unknown NACE activity']
eurostat = eurostat[~eurostat["sector"].isin(excluded)]

/var/folders/50/y5xpqyg11856c_nx11ysd4y00000gn/T/ipykernel_15947/2660573933.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eurostat["num_of_workers"] = eurostat["num_of_workers"] * 1000


In [39]:
# Pivot for aging ratios
eurostat = eurostat.pivot_table(
    index=["country", "sector"],
    columns=["age", "year"],
    values="num_of_workers",
    aggfunc="sum"
)

In [40]:
# Flatten MultiIndex and rename columns for merging
eurostat = eurostat.reset_index()
eurostat.columns = [
    f"{col[0]}_{col[1]}" if isinstance(col, tuple) else col
    for col in eurostat.columns
]
eurostat.columns = [col.rstrip("_") for col in eurostat.columns]

In [41]:
# Compute aging ratios
eurostat["aging_score_2014"] = eurostat["50 years or over_2014"] / eurostat["15 years or over_2014"] * 100
eurostat["aging_score_2024"] = eurostat["50 years or over_2024"] / eurostat["15 years or over_2024"] * 100
eurostat["aging_diff_2014_2024"] = (eurostat["aging_score_2024"] - eurostat["aging_score_2014"])

In [43]:
eurostat.fillna(0, inplace=True)

In [44]:
eurostat.to_csv("/Users/weronikadorociak/Documents/LSE/MY498 Capstone Project/Coding/Capstone_Project/Data_cleaning/data/eurostat_full.csv", index=False)